In [1]:
!pip install -r requirements.txt

In [2]:
# Install librosa (seems to come with ffmpeg as well):
!conda install -c conda-forge librosa -y

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
# Install ffmpeg:
# !conda install -c conda-forge ffmpeg -y

In [4]:
# Copy files over from S3
!aws s3 cp s3://video-wav2lip/lessonVid.mp4 sample_data/
!aws s3 cp s3://video-wav2lip/video.mp4 sample_data/
!ls sample_data

fatal error: An error occurred (404) when calling the HeadObject operation: Key "lessonVid.mp4" does not exist
download: s3://video-wav2lip/video.mp4 to sample_data/video.mp4   
audio.mp3  image.jpeg	    lessonVid.mp4  polly.wav
elon.mp4   lessonAudio.mp3  polly.mp3	   video.mp4


In [5]:
# Convert lesson video to audio
!ffmpeg -i sample_data/lessonVid.mp4 sample_data/lessonAudio.mp3 -y

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/ec2-user/anaconda3/envs/JupyterSystemEnv --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample 

In [6]:
!python inference.py --checkpoint_path checkpoints/wav2lip.pth --face "sample_data/video.mp4" --audio "sample_data/lessonAudio.mp3" --resize_factor 2

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 449
Extracting raw audio...
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/ec2-user/anaconda3/envs/JupyterSystemEnv --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7.

In [7]:
from moviepy.editor import *

# Merge original video (lessonVid.mp4) with Wav2Lip result video (results/result_voice.mp4)
clip1 = VideoFileClip(r'./sample_data/lessonVid.mp4')    # Use ./ instead of /
clip2 = VideoFileClip(r'./results/result_voice.mp4')

w1,h1 = clip1.size
w2,h2 = clip2.size
print(w1,h1)

desired_w = 0.25 * w1
resize_factor = desired_w / w2
clip2 = clip2.resize(resize_factor)
w_final,h_final = clip2.size
print(w_final,h_final)

1280 720
320 180


In [8]:
composite_clip = CompositeVideoClip([clip1,
                                 clip2.set_position(("right","bottom"))
                                 .set_start(0).crossfadein(1)])

# For quick experimentation: Get only the first 30s
# final_clip = trial_clip.subclip(t_end=(0,0,30))   

# Use a temp audio file if audio is not working
# It seems overriding and existing will result in second video not running correctly
composite_clip.write_videofile(r'./results/result_lesson.mp4', 
                     codec='libx264', 
                     audio_codec='aac', 
                     temp_audiofile='temp-audio.m4a', 
                     remove_temp=True
                     )
# final_clip.write_videofile(r'./results/result_voice.mp4')

chunk:   1%|▊                                                                | 55/4739 [00:00<00:08, 546.80it/s, now=None]

Moviepy - Building video ./results/result_lesson.mp4.
MoviePy - Writing audio in temp-audio.m4a


t:   0%|                                                                       | 4/6442 [00:00<04:11, 25.62it/s, now=None]

MoviePy - Done.
Moviepy - Writing video ./results/result_lesson.mp4



Moviepy - Done !
Moviepy - video ready ./results/result_lesson.mp4


In [9]:
!python email-aws.py lesson

cybersecma@gmail.com
results/result_lesson.mp4
DONE


Now, let's clean up our resources

In [10]:
!rm results/result_voice.mp4

In [11]:
!rm results/result_lesson.mp4

In [12]:
!aws s3 rm s3://video-wav2lip/video.mp4 

delete: s3://video-wav2lip/video.mp4


In [13]:
!aws s3 rm s3://video-wav2lip/lessonVid.mp4 

delete: s3://video-wav2lip/lessonVid.mp4
